In [ ]:
import pandas as pd
import numpy as np
import sktime
import seaborn as sns
import matplotlib.pyplot as plt
from convertcsv.import_preprocess_v4 import readcsvs
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sktime.classification.kernel_based import RocketClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.impute import KNNImputer
from convertcsv.get_all_metrics_with_tags import get_all_metrics_with_tags



In [ ]:
#What should we do here?

#Set up many different pipelines to compare.

#The things to vary, in order of significance/importance
"""
NaN treshold

KNN imptutation neigbours. Do research on this?
KNN weighting and distance metric
These can be researched separately by eliminating NaNs from a dataset, then using 
creating a loss function for the results of various types of KNN. Seems like a no brainer tbh.
How to do this? Simply removing all NaNs don't work. Use a similar, smaller data set.
Take a random subset of the thingies. Then clean it so there is no NaNs at all. Then 
set a random amount in random places to be NaNs. Aim for 25% perhaps
Actually first one has to remove those over the treshold, then look at the finished 
percentage. Emulate that. Write latex simultaneously.
"""
